In [7]:
import pandas as pd

df = pd.read_csv('df_standard1.csv')

# Initialize the lockscreen column with zeros
df['lockscreen'] = 0


# Group by 'trajectory_Id' and iterate over groups
for traj_id, group in df.groupby('trajectory_Id'):
    # Initialize lock screen state for each trajectory
    lock_screen = False
    
    # Iterate over rows in the group
    for index, row in group.iterrows():
        # Check if mapInteractions column has 'mapLog:isScreenLocked+True'
        if row['mapInteractions'] == 'mapLog:isScreenLocked+True':
            # Set lock_screen to True
            lock_screen = True
        # Check if mapInteractions column has 'mapLog:isScreenLocked+False'
        elif row['mapInteractions'] == 'mapLog:isScreenLocked+False':
            # Set lock_screen to False
            lock_screen = False
        # Assign 1 to lockscreen column if lock_screen is True
        if lock_screen:
            df.at[index, 'lockscreen'] = 1


# # Convert DateTime column to datetime format
# df['DateTime'] = pd.to_datetime(df['DateTime'])

# # Define a function to assign 'first' or 'second' based on DateTime
# def assign_half(group):
#     midpoint = group['DateTime'].iloc[len(group) // 2]
#     group['half'] = group['DateTime'].apply(lambda x: 'first' if x <= midpoint else 'second')
#     return group

# # Group by 'trajectory_Id' and apply the function to each group
# df = df.groupby('trajectory_Id').apply(assign_half)

# Save the DataFrame with the new column
df.to_csv("your_updated_dataset.csv", index=False)

C:\Users\xuzhengfang\AppData\Local\Temp\ipykernel_12136\500406029.py:42: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('trajectory_Id').apply(assign_half)


In [8]:
#____________________ Phase Time Calculation
import pandas as pd

df = pd.read_csv('your_updated_dataset.csv')
# Convert 'DateTime' column to datetime objects
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Initialize new columns for planning time and navigation time
df['active_time'] = 0
df['inactive_time'] = 0

# Group by 'trajectory_Id' and iterate over groups
for traj_id, group in df.groupby('trajectory_Id'):
    # Initialize variables to track previous time and index for planning and navigation phases
    prev_active_time = None
    prev_inactive_time = None
    prev_active_index = None
    prev_inactive_index = None
    
    # Iterate over rows in the group
    for index, row in group.iterrows():
        if row['lockscreen'] == 0:
            if prev_active_time is None:
                prev_active_time = row['DateTime']
                prev_inactive_index = index
            else:
                current_time = row['DateTime']
                active_time = (current_time - prev_active_time).total_seconds()
                df.at[index, 'active_time'] = active_time
                prev_active_time = current_time
                prev_active_index = index
            # Reset navigation variables when encountering planning phase
            prev_inactive_time = None
            
        elif row['lockscreen'] == 1:
            if prev_inactive_time is None:
                prev_inactive_time = row['DateTime']
                prev_inactive_index = index
            else:
                current_time = row['DateTime']
                prev_inactive_time = (current_time - prev_inactive_time).total_seconds()
                df.at[index, 'inactive_time'] = prev_inactive_time
                prev_inactive_time = current_time
                prev_inactive_index = index
            # Reset planning variables when encountering navigation phase
            prev_active_time = None


# Save the DataFrame with the updated column
df.to_csv('try.csv', index=False)

In [10]:
import pandas as pd

# Read your dataset into a pandas DataFrame
df = pd.read_csv('try.csv')

# Group by 'trajectory_Id' and 'half', then aggregate to calculate total active and inactive times
grouped_df = df.groupby(['trajectory_Id', 'half']).agg({
    'active_time': 'sum',
    'inactive_time': 'sum'
}).reset_index()

# Rename columns for clarity
grouped_df.rename(columns={'active_time': 'total_active_time', 'inactive_time': 'total_inactive_time'}, inplace=True)

# Save the grouped DataFrame to a new CSV file
grouped_df.to_csv("total_active_inactive_times.csv", index=False)


In [11]:
sum_df = grouped_df.groupby('half').agg({
    'total_active_time': 'sum',
    'total_inactive_time': 'sum'
}).reset_index()

# Save the resulting DataFrame to a new CSV file
sum_df.to_csv("sum_total_active_inactive_times.csv", index=False)

In [21]:
import pandas as pd

# Load the dataset
# Assuming the dataset is in a CSV file named 'data.csv'
df = pd.read_csv('try.csv')

# Filter for 'half' being 'first'
first_half_df = df[df['half'] == 'first']
# Count occurrences where 'mapInteractions' is 'mapLog:isScreenLocked+False' in 'first' half
first_half_count = (first_half_df['mapInteractions'] == 'mapLog:isScreenLocked+False').sum()

# Filter for 'half' being 'second'
second_half_df = df[df['half'] == 'second']
# Count occurrences where 'mapInteractions' is 'mapLog:isScreenLocked+False' in 'second' half
second_half_count = (second_half_df['mapInteractions'] == 'mapLog:isScreenLocked+False').sum()

print(f"The count of 'mapLog:isScreenLocked+False' in 'mapInteractions' when 'half' is 'first' is: {first_half_count}")
print(f"The count of 'mapLog:isScreenLocked+False' in 'mapInteractions' when 'half' is 'second' is: {second_half_count}")


The count of 'mapLog:isScreenLocked+False' in 'mapInteractions' when 'half' is 'first' is: 956
The count of 'mapLog:isScreenLocked+False' in 'mapInteractions' when 'half' is 'second' is: 945
